<a href="https://colab.research.google.com/github/reis-g/Machine-Learning/blob/main/Reconhecer_Objetos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow opencv-python opencv-python-headless

import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

dataset_path = '/content/drive/My Drive/Colab Notebooks/Produtos/dataset'

# Pré-processamento das imagens
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Carregar modelo pré-treinado
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Construir o modelo
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dense(len(train_data.class_indices), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Treinamento
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)


model.save('modelo_recomendacao.keras')


In [ ]:
# Extração de embeddings
embedding_model = Model(inputs=model.input, outputs=model.layers[-3].output)

# Gerar embeddings
def gerar_embedding(imagem):
    imagem = cv2.resize(imagem, (224, 224)) / 255.0
    imagem = np.expand_dims(imagem, axis=0)
    return embedding_model.predict(imagem)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import pickle

# Calcular similaridade
def recomendar(imagem_consulta, embeddings, nomes_produtos, top_n=5):
    consulta_embedding = gerar_embedding(imagem_consulta)
    similaridades = cosine_similarity(consulta_embedding, embeddings)
    indices = np.argsort(similaridades[0])[::-1][:top_n]
    return [(nomes_produtos[i], similaridades[0][i]) for i in indices]

# Simular embeddings armazenados
with open('embeddings.pkl', 'rb') as f:
    embeddings, nomes_produtos = pickle.load(f)

# Recomendação
imagem_teste = cv2.imread('/content/drive/My Drive/Colab Notebooks/Produtos/teste.jpg')
resultados = recomendar(imagem_teste, embeddings, nomes_produtos)

# Resultados
for produto, similaridade in resultados:
    print(f"Produto: {produto}, Similaridade: {similaridade:.2f}")


In [ ]:
# Gerar embeddings de todas as imagens no dataset
embeddings = []
nomes_produtos = []

for classe, indice in train_data.class_indices.items():
    classe_path = os.path.join(dataset_path, classe)
    for imagem_nome in os.listdir(classe_path):
        imagem = cv2.imread(os.path.join(classe_path, imagem_nome))
        if imagem is not None:
            embeddings.append(gerar_embedding(imagem))
            nomes_produtos.append(classe)

# Salvar embeddings
with open('embeddings.pkl', 'wb') as f:
    pickle.dump((embeddings, nomes_produtos), f)
